<table>
    <tr>
        <td><img src="https://s3.amazonaws.com/media-p.slid.es/uploads/1485763/images/9060062/Header.png" width="300"/></td>
        <td>&nbsp;</td>
        <td>
            <h1 style="font-size:200%;color:blue;text-align:center">    <FONT COLOR="blue">  Caso Final  </FONT>         </h1></td>         
        <td>
            <tp><p style="font-size:99%;text-align:center">PLN </p></tp>
            <tp><p style="font-size:115%;text-align:center">Pregrado MACC 2023-2</p></tp>
            <tp><p style="font-size:115%;text-align:center">Prof. Fabián Sánchez</p></tp>
        </td>
    </tr>
</table>

# <FONT SIZE=5 COLOR="green"> **Objetivo del estudio de caso y resultados de aprendizaje** </FONT>

En este estudio de caso se espera que el estudiante compare diversos métodos para resolver problemas de clasificación. Particularmente.

- Cargar y aplicar las técnicas de limpieza para normalizar un texto.

- Aplicar algunas técnicas de machine learning para clasificación

- Aplicar algunas estructuras de redes neuronales (MLP-LSTM) para clasificación.

- Comparar los resultados.

**objetivo**: dado un conjunto de datos de reviews de vinos, considerar las dos variables: *description* y *variety*. Determinar la variedad del vino a partir de descripción del mismo.



# <FONT SIZE=5 COLOR="green"> **Indicaciones** </FONT>

- Para entregar en grupos de máximo tres integrantes.

- Cargar en la plataforma *e-aulas* los archivos: *.ipynb* y *.pdf*

- Fecha de entrega el día **martes 28 de Noviembre 6:00 p.m.**

- Preparar una presentación de los resultados del caso para el día **miércoles 29 de Noviembre de 2023** en la sesión de clase.

- Entrega de notas: viernes **1 de diciembre**



In [1]:
# ! pip install keras tensorflow transformers sentencepiece wordcloud stop-words -q

In [3]:
# Cargamos las librerias necesarias
import pandas as pd
import pickle
import plotly.express as px
import re
import warnings
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from wordcloud import WordCloud
warnings.filterwarnings(action='ignore')

# <FONT SIZE=5 COLOR="bronw"> **Punto 1. Carga y Exploración de los datos** </FONT>

## <FONT SIZE=4 COLOR="blue"> **1. Carga y Exploración de los datos**

Utilizando la librería Pandas, se realiza la carga del conjunto de datos desde el archivo `Wine_Reviews.csv`. Posteriormente, se imprime una descripción breve y esencial del conjunto de datos mediante data.info() y data.describe(). Permitiendo tener una visión general de la estructura de los datos, incluyendo la cantidad de registros, el tipo de datos en cada columna y estadísticas descriptivas relevantes para columnas numéricas, como la media, desviación estándar, y valores mínimos y máximos.

In [10]:
data = pd.read_csv('data/Wine_Reviews.csv')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             129971 non-null  int64  
 1   country                129908 non-null  object 
 2   description            129971 non-null  object 
 3   designation            92506 non-null   object 
 4   points                 129971 non-null  int64  
 5   price                  120975 non-null  float64
 6   province               129908 non-null  object 
 7   region_1               108724 non-null  object 
 8   region_2               50511 non-null   object 
 9   taster_name            103727 non-null  object 
 10  taster_twitter_handle  98758 non-null   object 
 11  title                  129971 non-null  object 
 12  variety                129970 non-null  object 
 13  winery                 129971 non-null  object 
dtypes: float64(1), int64(2), object(11)


In [6]:
data.describe()

,Unnamed: 0,points,price
count,129971.000000,129971.000000,120975.000000
mean,64985.000000,88.447138,35.363389
std,37519.540256,3.039730,41.022218
min,0.000000,80.000000,4.000000
25%,32492.500000,86.000000,17.000000
50%,64985.000000,88.000000,25.000000
75%,97477.500000,91.000000,42.000000
max,129970.000000,100.000000,3300.000000


In [7]:
data.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


**Agregar una buena descripción de los datos**

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla et rhoncus ligula, varius gravida arcu. Proin vitae massa pharetra, commodo mi quis, bibendum sem. Nunc aliquet magna sit amet massa auctor, et pulvinar leo consectetur. Mauris ullamcorper mi non enim condimentum semper. Integer tempus, orci id ultricies euismod, nunc dui dictum velit, a venenatis tellus nisl vel tellus. Suspendisse vulputate rutrum porttitor. Vestibulum sed leo blandit, posuere ligula vel, varius turpis. Proin erat ipsum, rutrum in orci nec, pharetra blandit nunc. Cras magna orci, dapibus non porttitor et, auctor et lacus. Nunc quis libero neque. 

Nulla dui dolor, pretium vitae malesuada at, ultrices eget nibh. Aenean dapibus iaculis orci, ornare suscipit ex gravida vitae. Donec laoreet eleifend ligula, eu dignissim urna lacinia ac. Donec euismod turpis ex, nec convallis lectus lacinia nec. Fusce vel ipsum ut velit aliquam auctor. Curabitur commodo mollis sem. Proin scelerisque erat sed condimentum imperdiet. Mauris nisi neque, bibendum vitae placerat nec, eleifend nec urna. Lorem ipsum dolor sit amet, consectetur adipiscing elit. 

Concluir con que se eliminaran las demás variables, diferentes a `description` y `variety`.

In [12]:
data = data.drop(columns=['Unnamed: 0', 'country', 'designation', 'points',
                          'price', 'province', 'region_1', 'region_2', 'taster_name',
                          'taster_twitter_handle', 'title', 'winery'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   description  129971 non-null  object
 1   variety      129970 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


3. Realizar una tabla de frecuencias y gráfico de barras de la variable *variety*.

4. Del punto anterior seleccionar el top 10. Es decir, las 10 variedades de vino con más registros.

5. De cada una de las 10 variedades de vino, seleccionar 2000 registros de manera aleatoria. Es decir, la base final quedará constituida por 20.000 registros.

6. Realizar el proceso de normalización (limpieza) de la variable *description*.Tenga en cuenta guardar el pre-procesado en un archivo *pickle*.

7. Hacer la nube de palabras de la variable *description*.

# <FONT SIZE=5 COLOR="bronw"> **Punto 2. Modelos ML and ANN** </FONT>

1. Seleccione dos modelos de machine learning para resolver el problema de clasificación.

2. Aplique MLP (multilayer perceptron) con *Tensorflow-Keras* para generar el modelo.

3. Aplique una estructura de LSTM con *Tensorflow-Keras* para generar el modelo.

4. Evalúe y compare los resultados de los modelos

5. Concluya

# <FONT SIZE=5 COLOR="bronw"> **Punto 3. Topic Modeling** </FONT>

- Utilice el metodo LDA visto en clase para generar clustering a partir de la variable *description*.
- Use la visualización LDA para explorar las palabras más relevantes de los grupos.
- Concluya